# Group Details

## Group Name: GLS  

### Student 1: Fabio D'Agostino

### Student 2:

### Student 3:

# Loading Data and Preliminaries

In [2]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [3]:
def load_array(filename, task):
    datapoint = np.load(filename)
    if task == 'task 1':
        initial_state = datapoint['initial_state']
        terminal_state = datapoint['terminal_state']
        return initial_state, terminal_state
    elif task == 'task 2' or task == 'task 3':
        whole_trajectory = datapoint['trajectory']
        # change shape: (num_bodies, attributes, time) ->  num_bodies, time, attributes
        whole_trajectory = np.swapaxes(whole_trajectory, 1, 2)
        initial_state = whole_trajectory[:, 0]
        target = whole_trajectory[:, 1:, 1:]  # drop the first timepoint (second dim) and mass (last dim) for the prediction task
        return initial_state, target
    else:
        raise NotImplementedError("'task' argument should be 'task 1', 'task 2' or 'task 3'!")


In [4]:
"""
This cell gives an example of loading a datapoint with numpy for task 1.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
terminal_state: shape (n_bodies, [x, y])

"""

example = load_array('data/task 1/train/trajectory_8.npz', task='task 1')

initial_state, terminal_state = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {terminal_state.shape}')

body_idx = 2
print(f'The initial x-coordinate of the body with index {body_idx} in this trajectory was {initial_state[body_idx, 1]}')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 2)
The initial x-coordinate of the body with index 2 in this trajectory was -4.337538425576772


In [5]:
type(initial_state)

numpy.ndarray

In [6]:
#Extract training data
X_train_list = []
y_train_list = []
for i in range(900):
    initial_state, terminal_state = load_array('data/task 1/train/trajectory_' + str(i) + '.npz', task='task 1')
    X_train_list.append(initial_state)
    y_train_list.append(terminal_state)

In [8]:
#Extract training data
X_val_list = []
y_val_list = []
for i in range(100):
    initial_state, terminal_state = load_array('data/task 1/test/trajectory_' + str(i+900) + '.npz', task='task 1')
    X_val_list.append(initial_state)
    y_val_list.append(terminal_state)

In [79]:
example[1]

array([[ 0.28370052, -9.25453722],
       [-4.51021897, -5.13595325],
       [ 2.83844977,  5.36202729],
       [ 2.12939268,  4.64550475],
       [-1.59977861, -8.64922402],
       [ 2.4614544 , -7.55374664],
       [-4.32206998, -3.49588612],
       [-6.08779906,  6.41931202]])

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GATConv
from torch_geometric.loader import DataLoader

# Define the GNN model
class GNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GNNModel, self).__init__()
        self.conv1 = GATConv(input_size, hidden_size)
        self.conv2 = GATConv(hidden_size, output_size)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Convert data to PyTorch Geometric format (no forces between objects)
#def convert_to_graph_data(X, y):
#    n_bodies, _ = X.shape
#    edge_index = []
#    for i in range(n_bodies):
#        for j in range(i + 1, n_bodies):
#            edge_index.append([i, j])
#            edge_index.append([j, i])
#    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
#    x = torch.tensor(X, dtype=torch.float)
#    y = torch.tensor(y, dtype=torch.float)
#    data = Data(x=x, edge_index=edge_index, y=y)
#    return data

# Convert data to PyTorch Geometric format (with forces between objects)
def convert_to_graph_data(X, y):
    n_bodies, _ = X.shape
    edge_index = []
    edge_attr = []
    
    # Calculate forces between objects
    for i in range(n_bodies):
        for j in range(i + 1, n_bodies):
            force = X[i, 1:3] - X[j, 1:3]  # Force vector [x1-x2, y1-y2]
            edge_attr.append(force)
            edge_index.append([i, j])
            edge_attr.append(-force)  # Negate the force for the opposite direction
            edge_index.append([j, i])
    
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    x = torch.tensor(X, dtype=torch.float)
    y = torch.tensor(y, dtype=torch.float)
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
    return data

# Preprocessing step
X_data_train = X_train_list  # List of numpy arrays, each with shape (n_objects, 5)
y_data_train = y_train_list  # List of numpy arrays, each with shape (n_objects, 2)

# Convert training data to tensors and create DataLoader
training_dataset = []
for i in range(len(X_data_train)):
    data = convert_to_graph_data(X_data_train[i], y_data_train[i])
    training_dataset.append(data)
training_loader = DataLoader(training_dataset, batch_size=1, shuffle=True)


# Preprocessing step
X_data_val = X_val_list  # List of numpy arrays, each with shape (n_objects, 5)
y_data_val = y_val_list  # List of numpy arrays, each with shape (n_objects, 2)

# Convert evaluation data to tensors and create DataLoader
val_dataset = []
for i in range(len(X_data_val)):
    data = convert_to_graph_data(X_data_val[i], y_data_val[i])
    val_dataset.append(data)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# Model configuration
input_size = 5  # Features per object (mass, x, y, v_x, v_y)
hidden_size = 16
output_size = 2  # Final position (x, y)

# Instantiate the model
model = GNNModel(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


C:\Users\dagos\AppData\Local\Temp\ipykernel_18180\395197383.py:51: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  edge_attr = torch.tensor(edge_attr, dtype=torch.float)


In [22]:
for data in training_loader:
    print(data)
    break

DataBatch(x=[9, 5], edge_index=[2, 72], edge_attr=[72, 2], y=[9, 2], batch=[9], ptr=[2])


In [23]:
# Training step
def train(model, loader, optimizer, criterion, num_epochs):
    model.train()
    
    for epoch in range(num_epochs):
        total_loss = 0
        
        for data in loader:
            optimizer.zero_grad()
            out = model(data.x, data.edge_index, data.batch)
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(loader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")

# Evaluation step
def evaluate(model, loader):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for data in loader:
            out = model(data.x, data.edge_index, data.batch)
            loss = criterion(out, data.y)
            total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    return avg_loss

# Train the model
num_epochs = 100
train(model, training_loader, optimizer, criterion, num_epochs)

# Evaluate the model
loss = evaluate(model, training_loader)
print(f"Final Evaluation Loss: {loss:.4f}")

# Example prediction
#test_X = X_val_list  # Shape: (n_objects, 5)
#test_data, _ = convert_to_graph_data(test_X, np.zeros((test_X.shape[0], 2)))
#test_batch = torch.tensor([0] * len(test_data.x), dtype=torch.long)  # Assuming all objects belong to the same system
#batched_test_data = Batch.from_data_list([test_data])
#prediction = model(batched_test_data.x, batched_test_data.edge_index, batched_test_data.batch)
#print("Predicted terminal positions:", prediction.tolist())

Epoch 1/100, Loss: 20.4041
Epoch 2/100, Loss: 19.0677
Epoch 3/100, Loss: 19.0146
Epoch 4/100, Loss: 18.9757
Epoch 5/100, Loss: 18.9589
Epoch 6/100, Loss: 18.9462
Epoch 7/100, Loss: 18.9344
Epoch 8/100, Loss: 18.9161
Epoch 9/100, Loss: 18.9121
Epoch 10/100, Loss: 18.9060
Epoch 11/100, Loss: 18.9048
Epoch 12/100, Loss: 18.8970
Epoch 13/100, Loss: 18.8971
Epoch 14/100, Loss: 18.8948
Epoch 15/100, Loss: 18.8904
Epoch 16/100, Loss: 18.8853
Epoch 17/100, Loss: 18.8919
Epoch 18/100, Loss: 18.8911
Epoch 19/100, Loss: 18.8910
Epoch 20/100, Loss: 18.8847
Epoch 21/100, Loss: 18.8914


KeyboardInterrupt: 

In [113]:
###TRASH LSTM

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        #self.hidden_size = hidden_size
        #self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.relu = nn.ReLU()
        self.linear_1 = nn.Linear(input_size, 2048)
        self.linear_2 = nn.Linear(2048, 1024)
        self.linear_3 = nn.Linear(1024, output_size)

    def forward(self, x):
        #_, (h, _) = self.lstm(x)
        x = self.linear_1(x)
        x = self.relu(x)
        x= self.linear_2(x)
        x = self.relu(x)
        x= self.linear_3(x)
        return x

# Preprocessing step
X_data = X_train

y_data = y_train

# Convert data to tensors
X = torch.tensor(X_data, dtype=torch.float)
y = torch.tensor(y_data, dtype=torch.float)

# Model configuration
input_size = X_data.shape[1]
hidden_size = 16
output_size = y_data.shape[1]

# Instantiate the model
model = LSTMModel(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training step
def train(model, X, y, optimizer, criterion, num_epochs):
    model.train()
    
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        out = model(X)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 100 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

# Evaluation step
def evaluate(model, X, y):
    model.eval()
    out = model(X)
    loss = criterion(out, y)
    return loss.item()

# Train the model
num_epochs = 1000
train(model, X, y, optimizer, criterion, num_epochs)

# Evaluate the model
loss = evaluate(model, X, y)
print(f"Final Evaluation Loss: {loss}")

# Example prediction
test_data = np.array([[9.0, 4.0, 4.0, 9.0, 9.0]])
test_X = torch.tensor(test_data, dtype=torch.float)
prediction = model(test_X)
print("Predicted terminal positions:", prediction.tolist())


Epoch 100/1000, Loss: 6.917230606079102
Epoch 200/1000, Loss: 5.944492816925049
Epoch 300/1000, Loss: 5.496774196624756
Epoch 400/1000, Loss: 5.317768096923828
Epoch 500/1000, Loss: 5.1487603187561035
Epoch 600/1000, Loss: 4.953726291656494
Epoch 700/1000, Loss: 4.842867851257324
Epoch 800/1000, Loss: 4.763996124267578
Epoch 900/1000, Loss: 4.586665630340576
Epoch 1000/1000, Loss: 4.3971781730651855
Final Evaluation Loss: 4.415316581726074
Predicted terminal positions: [[-0.4754807651042938, 0.8647503852844238]]


# Data Handling and Preprocessing

In [10]:
#todo

# Model Implementation

In [11]:
#todo

# Model Training

In [12]:
#todo

# Evaluation

In [13]:
#todo